In [1]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (0.24.2)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pyarrow

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from numpy import nan as NA
import os
import gc
from pandas.tseries.offsets import MonthEnd

from azure.storage.blob import BlockBlobService
import pyarrow.parquet as pq
from io import StringIO
from io import BytesIO

In [4]:
YearMonth = '2018-06-01'

In [5]:
#Azure account credentials
azure_storage_account_name = "agedatasciencecocstorage"
azure_storage_account_key = "V/FXG+lIWtYc854wbRA8BxGCLGJijS0NDwLvNHfmAxqRopAL3WHWGIQoHxpArG9lZoX+iY+dvtPYoB379w4MIw=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [6]:
# First, we need to connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
# There are also methods to list containers and blobs
#containers = blob_service.list_containers()
blobs = blob_service.list_blobs('banking-dm')

In [7]:
# List the blobs in the container.
print("\nList blobs in the container")
#generator = block_blob_service.list_blobs(container_name)
for blob in blobs:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: ailiev/Balances_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-06-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-01-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-02-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-03-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-04-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-05-01.parquet.gzip
	 Blob name: ailiev/MCIF_2018-06-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-05-01.parquet.gz

### Read Data

In [8]:
container_name = "banking-dm"
# blob_name_parquet = 'data/Customer_information_' + YearMonth + '.parquet.gzip'

In [9]:
for i in range(1, 4):
    print(i)
    blob_name_parquet = 'ailiev/MCIF_2018-0' + str(i) + '-01.parquet.gzip'
    byte_stream = BytesIO()
    try:
       blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet, stream=byte_stream)
       file = pq.read_table(source=byte_stream).to_pandas()
       # Do work on df ...
    finally:
       # Add finally block to ensure closure of the stream
       byte_stream.close()
    if i==1:
        df = file
    else:
        df = pd.concat([df, file], ignore_index=True)
    del file
    gc.collect()

1
2
3


In [10]:
df.shape

(300358, 278)

In [11]:
df.size

83499524

In [12]:
df.head()

,YearMonth,Customer_ID,CUSTOMER_TYPE,CUSTOMER_CATEGORY,GENDER,MARITAL_STATUS,CUSTOMER_ECO_BRANCH,SECTOR,AGE_MONTHS,CNT_AL_PROMOTION,...,PCT_TNAM_DT,PCT_NUTN_CC,PCT_NUTN_DC,PCT_NUTN_LD,PCT_TNAM_CC,PCT_TNAM_DC,PCT_TNAM_LD,PCT_DAYS_TR_CC,PCT_DAYS_TR_DC,PCT_DAYS_TR_LD
0,2018-01-01,1,Individual,None,F,Divorced,BR160,Arts and Communications,556,NaN,...,1.000000,NaN,0.320000,0.680000,NaN,0.332096,0.667904,NaN,0.333333,0.833333
1,2018-01-01,2,Professional,None,M,Married,BR09,Finance,518,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN
2,2018-01-01,3,Individual,None,M,Married,BR145,Manufacturing,308,NaN,...,0.678937,NaN,0.387097,0.612903,NaN,0.220054,0.779946,NaN,0.476190,0.714286
3,2018-01-01,4,Small Business,Retail,None,None,BR14,Health and Science,482,NaN,...,0.870951,NaN,0.968750,0.031250,NaN,0.870951,0.129049,NaN,1.000000,0.050000
4,2018-01-01,5,Individual,None,M,Single,BR171,Agriculture and Food,475,NaN,...,1.000000,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN


In [13]:
# df[df['YearMonth']=='2018-02-01'].head()

In [14]:
#aggreg_prod = pd.pivot_table(df, index=['Customer_ID'])

In [15]:
#aggreg_prod.head()

In [16]:
#aggreg_prod.shape

In [17]:
#sample= df[(df['Customer_ID']<=5) | (df['Customer_ID']>=100140)]

In [18]:
# output = StringIO()
# output = sample.to_csv (index_label="idx", encoding = "utf-8")
# blob_service.create_blob_from_text('banking-dm', 'ailiev/sample.csv', output)

### Aggregation

In [19]:
demographic_columns = ['YearMonth', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY', 'GENDER', 'MARITAL_STATUS', 'CUSTOMER_ECO_BRANCH', 'SECTOR', 'AGE_MONTHS']

In [20]:
columns_to_aggregate = [col for col in df.columns if col not in demographic_columns]

In [21]:
#columns_to_aggregate

In [22]:
not_avg_cols = ['CLOSED_MONTHS_MIN', 'DC_MATURE_MONTHS_MAX', 'MATURE_MONTHS_MAX', 'MATURE_MONTHS_MIN', 'MONTHS_TILL_EXP_MAX', 'MONTHS_TILL_EXP_MIN']

In [23]:
avg_cols = [col for col in columns_to_aggregate if col not in not_avg_cols]

In [24]:
len(avg_cols)

264

In [25]:
len(columns_to_aggregate)

270

In [26]:
# df = df[columns_to_aggregate]

In [27]:
df.shape

(300358, 278)

In [28]:
# [avg_cols]: [np.mean]

In [29]:
aggregated_avg = pd.pivot_table(df[avg_cols], index=['Customer_ID'])

In [30]:
aggregated_avg.reset_index(inplace=True)
aggregated_avg.head()

,Customer_ID,AT_LEAST_ONE_ACTIVE,AVG_DEPOSITS_minus_LOANS,BAL_AVG_AL_PROMOTION,BAL_AVG_AL_STANDARD,BAL_AVG_CA_PAYROLL,BAL_AVG_CA_STANDARD,BAL_AVG_CC_MASTERCARD_GOLD,BAL_AVG_CC_MASTERCARD_PLATINUM,BAL_AVG_CC_MASTERCARD_STANDARD,...,TNAM_LD_TRANSFERS,TNAM_LD_WITHDRAWALS,TOTAL_DIFF_CAT,TOTAL_LOANS_BAL,TOTAL_LOANS_CNT,TOTAL_NUTN,TOTAL_PRODUCTS,TOTAL_SAVING_BAL,TOTAL_SAVING_CNT,TOTAL_TNAM
0,1,0.666667,157.725083,NaN,NaN,NaN,56.707875,NaN,NaN,NaN,...,337.517500,374.835,2.0,-119.919833,1.0,24.000000,2.0,37.805250,1.0,1026.335000
1,2,1.000000,0.000000,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,...,0.000000,NaN,1.0,0.000000,0.0,1.000000,1.0,0.000000,1.0,161.413333
2,3,1.000000,1616.158533,NaN,NaN,NaN,1616.158533,NaN,NaN,NaN,...,880.588333,1154.880,1.0,0.000000,0.0,36.666667,1.0,1616.158533,1.0,3606.596667
3,4,1.000000,-271.317717,NaN,NaN,312.92835,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,479.936617,3.0,38.000000,4.0,208.618900,1.0,3869.115000
4,5,1.000000,-836.073333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,836.073333,1.0,31.333333,2.0,0.000000,1.0,1923.821667


In [31]:
aggregated_avg.shape

(100215, 264)

In [32]:
aggregated_min_max = pd.pivot_table(df, index=['Customer_ID'], 
                             aggfunc={'CLOSED_MONTHS_MIN': min, 
                                      'MATURE_MONTHS_MIN': min, 
                                      'MONTHS_TILL_EXP_MIN': min,
                                      'DC_MATURE_MONTHS_MAX': max, 
                                      'MATURE_MONTHS_MAX': max, 
                                      'MONTHS_TILL_EXP_MAX': max})

In [33]:
aggregated_min_max.reset_index(inplace=True)
aggregated_min_max.head()

,Customer_ID,CLOSED_MONTHS_MIN,DC_MATURE_MONTHS_MAX,MATURE_MONTHS_MAX,MATURE_MONTHS_MIN,MONTHS_TILL_EXP_MAX,MONTHS_TILL_EXP_MIN
0,1,-1,-1,54,0,-1,-1
1,2,-1,59,215,213,-1,-1
2,3,-1,-1,229,227,-1,-1
3,4,-1,93,120,0,17,-18
4,5,-1,-1,171,68,2,-1


In [34]:
aggregated_min_max.shape

(100215, 7)

In [35]:
demographic_columns.append('Customer_ID')

In [36]:
demographic_columns.reverse()

In [37]:
#df[df['YearMonth']=='2018-02-01'][demographic_columns].head()

In [38]:
avg_mcif = df[df['YearMonth']=='2018-03-01'][demographic_columns]

In [39]:
del avg_mcif['YearMonth']

In [40]:
avg_mcif.head()

,Customer_ID,AGE_MONTHS,SECTOR,CUSTOMER_ECO_BRANCH,MARITAL_STATUS,GENDER,CUSTOMER_CATEGORY,CUSTOMER_TYPE
200143,1,558,Arts and Communications,BR160,Divorced,F,None,Individual
200144,2,519,Finance,BR09,Married,M,None,Professional
200145,3,310,Manufacturing,BR145,Married,M,None,Individual
200146,4,484,Health and Science,BR14,None,None,Retail,Small Business
200147,5,477,Agriculture and Food,BR171,Single,M,None,Individual


In [41]:
avg_mcif = avg_mcif.merge(aggregated_avg, on='Customer_ID')

In [42]:
avg_mcif = avg_mcif.merge(aggregated_min_max, on='Customer_ID')

In [43]:
avg_mcif.shape

(100215, 277)

In [44]:
avg_mcif['AVG_PERIOD'] = '2018-01--2018-03'

In [45]:
avg_mcif.head()

,Customer_ID,AGE_MONTHS,SECTOR,CUSTOMER_ECO_BRANCH,MARITAL_STATUS,GENDER,CUSTOMER_CATEGORY,CUSTOMER_TYPE,AT_LEAST_ONE_ACTIVE,AVG_DEPOSITS_minus_LOANS,...,TOTAL_SAVING_BAL,TOTAL_SAVING_CNT,TOTAL_TNAM,CLOSED_MONTHS_MIN,DC_MATURE_MONTHS_MAX,MATURE_MONTHS_MAX,MATURE_MONTHS_MIN,MONTHS_TILL_EXP_MAX,MONTHS_TILL_EXP_MIN,AVG_PERIOD
0,1,558,Arts and Communications,BR160,Divorced,F,None,Individual,0.666667,157.725083,...,37.805250,1.0,1026.335000,-1,-1,54,0,-1,-1,2018-01--2018-03
1,2,519,Finance,BR09,Married,M,None,Professional,1.000000,0.000000,...,0.000000,1.0,161.413333,-1,59,215,213,-1,-1,2018-01--2018-03
2,3,310,Manufacturing,BR145,Married,M,None,Individual,1.000000,1616.158533,...,1616.158533,1.0,3606.596667,-1,-1,229,227,-1,-1,2018-01--2018-03
3,4,484,Health and Science,BR14,None,None,Retail,Small Business,1.000000,-271.317717,...,208.618900,1.0,3869.115000,-1,93,120,0,17,-18,2018-01--2018-03
4,5,477,Agriculture and Food,BR171,Single,M,None,Individual,1.000000,-836.073333,...,0.000000,1.0,1923.821667,-1,-1,171,68,2,-1,2018-01--2018-03


In [46]:
def write_pandas_dataframe_to_blob(blob_service, df, container_name, blob_name):
    """ Write Pandas dataframe to blob storage """
    buffer = BytesIO()
    df.to_parquet(buffer)
    blob_service.create_blob_from_bytes(
        container_name=container_name, blob_name=blob_name, blob=buffer.getvalue()
    )

In [47]:
write_to_blob_parquet = 'ailiev/3M_AVG_MCIF_2018-01__2018-03.parquet.gzip'

In [48]:
# Write to blob using pyarrow
write_pandas_dataframe_to_blob(blob_service, avg_mcif, 'banking-dm', write_to_blob_parquet)

In [49]:
del avg_mcif
gc.collect()

29